In [1]:
import pandas as pd

In [2]:
import numpy as np

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import warnings

warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 100)

# Import csv file
data = pd.read_csv('final_data.csv', encoding='unicode_escape', index_col=0)


In [4]:
data.head(5)

,beer_name,user,aroma,appearance,flavor,mouthfeel,overall,rating,date
1,San Miguel,ResinousMaestro55(268),5,3,5,3,9,2.5,2021-01-15
2,San Miguel,"nimbleprop(11,986)",5,3,5,2,10,2.5,2020-02-09
3,San Miguel,"troopie(4,238)",2,3,3,1,4,1.3,2019-06-13
4,San Miguel,BubbleTamer35(1),2,4,3,4,6,1.9,2019-05-28
5,San Miguel,"vladeath(1,972)",4,2,4,2,9,2.1,2019-08-13


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91776 entries, 1 to 91776
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   beer_name   91776 non-null  object 
 1   user        91776 non-null  object 
 2   aroma       91776 non-null  int64  
 3   appearance  91776 non-null  int64  
 4   flavor      91776 non-null  int64  
 5   mouthfeel   91776 non-null  int64  
 6   overall     91776 non-null  int64  
 7   rating      91776 non-null  float64
 8   date        91776 non-null  object 
dtypes: float64(1), int64(5), object(3)
memory usage: 7.0+ MB


In [8]:
# split the date data (year, month, day)
year_score = data.copy()
year_score['year'] = year_score['date'].apply(lambda x : x.split('-')[0])
year_score['month'] = year_score['date'].apply(lambda x : x.split('-')[1])

year_score.head()

,beer_name,user,aroma,appearance,flavor,mouthfeel,overall,rating,date,year,month
1,San Miguel,ResinousMaestro55(268),5,3,5,3,9,2.5,2021-01-15,2021,01
2,San Miguel,"nimbleprop(11,986)",5,3,5,2,10,2.5,2020-02-09,2020,02
3,San Miguel,"troopie(4,238)",2,3,3,1,4,1.3,2019-06-13,2019,06
4,San Miguel,BubbleTamer35(1),2,4,3,4,6,1.9,2019-05-28,2019,05
5,San Miguel,"vladeath(1,972)",4,2,4,2,9,2.1,2019-08-13,2019,08


In [10]:
num_beer = year_score.copy()

# the number of reviews (ttmp = num_beer)
num_beer['beer_name'].value_counts().head()

Guinness Draught                        5628
Budweiser                               4361
Pilsner Urquell                         4251
Heineken                                4136
Newcastle Brown Ale (Non-US Version)    3933
Name: beer_name, dtype: int64

In [11]:
#calculate the each of beer score by date (year)
df_year = num_beer['rating'].groupby(num_beer['year']).mean()
df_year = pd.DataFrame(data=df_year, columns = ['rating'])

#change index to column
df_year['year'] = df_year.index
df_year

,rating,year
year,,
2000,2.756356,2000
2001,2.675696,2001
2002,2.580451,2002
2003,2.545760,2003
2004,2.554627,2004
2005,2.571141,2005
2006,2.563824,2006
2007,2.558186,2007
2008,2.572839,2008


In [19]:
num_beer.head()

# beer rating per year
beer_by_year = num_beer['rating'].groupby([num_beer['beer_name'],num_beer['year']]).mean()
beer_by_year.head(20)

beer_by_year = pd.DataFrame(data=beer_by_year, columns = ['rating'])
beer_by_year

beer_by_year.reset_index(level=['beer_name','year'], inplace=True)
beer_by_year.head()

# save as csv
beer_by_year.to_csv('beer_score_by_year.csv', encoding='utf-8')

In [8]:
# Delete duplicated "beer name"
beer_data = data.copy()
beer_data = beer_data[['beer_name']]

beer_data.drop_duplicates(keep='first', inplace=True)
cols = ['aroma', 'appearance','flavor','mouthfeel','overall']

# Create 5 categories for column name
for col in cols:
    beer_data[col] = ''
beer_data.head(5)

,beer_name,aroma,appearance,flavor,mouthfeel,overall
1,San Miguel,,,,,
192,Newcastle Brown Ale (Non-US Version),,,,,
4125,Rickard's Red,,,,,
4621,Paradox Beer Candid Kaiser - American Style Pa...,,,,,
4628,Maclay's Traditional Pale Ale,,,,,


In [10]:
# Calculate average score for each columns
for i in range(len(beer_data)):
   
    # collect beer name
    beer = beer_data['beer_name'].iloc[i]

    # the total number of reviewers
    length = len(data[data['beer_name']==beer])
    # calculate the each columns 
    for col in cols:
        # total score for each columns 
        col_sum = data[data['beer_name']==beer][col].sum()
        # average score for each columns
        beer_data[col].iloc[i] = col_sum/length

beer_data.to_csv('beer_score.csv', encoding='utf-8')
beer_data.head(5)

,beer_name,aroma,appearance,flavor,mouthfeel,overall
1,San Miguel,3.63351,2.5445,3.82199,2.2199,7.60733
192,Newcastle Brown Ale (Non-US Version),5.4152,3.41114,5.92525,3.01831,12.107
4125,Rickard's Red,4.77218,3.25202,5.12097,2.6875,10.2823
4621,Paradox Beer Candid Kaiser - American Style Pa...,8,3.85714,7.14286,4.14286,15
4628,Maclay's Traditional Pale Ale,3.63303,2.33945,4.06422,2.20183,8
